In [114]:
from ps4_utils import load_data,load_experiment
from ps4_utils import AbstractGenerativeModel
from ps4_utils import save_submission
from scipy.misc import logsumexp
import numpy as np
data_fn = "datasets-ps4.h5"
MAX_OUTER_ITER = 15

In [115]:
class MixtureModel(AbstractGenerativeModel):
    def __init__(self, CLASSES, NUM_FEATURES, NUM_MIXTURE_COMPONENTS, MAX_ITER=50, EPS=10**(-7)):
        AbstractGenerativeModel.__init__(self, CLASSES, NUM_FEATURES)
        self.num_mixture_components = NUM_MIXTURE_COMPONENTS # list of num_mixture_components (length num_classes)
        self.max_iter = MAX_ITER # max iterations of EM
        self.epsilon = EPS # help with stability, to be used according to hint given at end of pset4.pdf
        self.params = { # lists of length CLASSES
            'pi': [np.repeat(1.0/k,k) for k in self.num_mixture_components], # with pi_c for each class
            # Original: np.zeros()
            'theta': [np.random.rand(self.num_features,k) for k in self.num_mixture_components], # with theta_c for each class
        }
    def pack_params(self, X, class_idx):
        pi,theta = self.fit(X[class_idx],class_idx) # fit parameters
        self.params['pi'][class_idx] = pi # update member variable pi
        self.params['theta'][class_idx] = theta #update member variable theta
        
    #make classification based on which mixture model gives higher probability to generating point xi
    def classify(self, X):
        P = list()
        pi = self.params['pi']
        theta = self.params['theta']
        for c in range(self.num_classes):
            _,Pc = self.findP(X, pi[c], theta[c])
            P.append(Pc)
        return np.vstack(P).T.argmax(-1) # np.array of class predictions for each data point in X

    # --- E-step
    def updateLatentPosterior(self, X, pi, theta, num_mixture_components): # update the latent posterior
        # YOUR CODE HERE
        # --- gamma: np.array (matrix)
        # ---        shape: number of data points in X (where X consists of datapoints from class c) by NUM_MIXTURE_COMPONENTS[c]
        N = X.shape[0]
        gamma = np.zeros([N, num_mixture_components])
#         for i in range(N):
#             log_A = np.zeros([num_mixture_components])
#             for c in range(num_mixture_components):
#                 log_A[c] = np.dot(X[i], np.log(theta.T[c])) + np.dot(1 - X[i], np.log(1-theta[c]))
#             A = np.exp(log_A)/np.sum(np.exp(log_A))
#             gamma[i] = A
        for c in range(num_mixture_components):
            theta_c = theta[:, c].clip(min = 1e-7, max = 1.0 - 1e-7)
#             theta_c = theta[:, c]
            log_gamma = np.log(np.repeat(pi[c], N)) + np.dot(X, np.log(theta_c)) + np.dot(1.0 - X, np.log(1 - theta_c))
            gamma[:, c] = log_gamma
        gamma = gamma - np.mean(gamma)
        gamma = np.exp(gamma)
        gamma = np.divide(gamma, np.repeat(np.sum(gamma, axis=1).reshape([N, 1]), num_mixture_components, axis=1))
#         for i in range(N):
#             s = np.sum(gamma[i, :])
#             if s - 1.0 >= 1.0e-5 or s - 1.0 <= -1.0e-5:
#                 print("gamma line sum not zero: line {}, sum: {}".format(i, s))
        return gamma
    # --- M-step (1)
    @staticmethod
    def updatePi(gamma): #update the pi component using the posteriors (gammas)
        # YOUR CODE HERE
        # --- pi_c: class specific pi, np.array (vector)
        # ---        shape: NUM_MIXTURE_COMPONENTS[c]
        pi_c = np.sum(gamma, axis=0)
        return pi_c
    # -- M-step (2)
    @staticmethod
    def updateTheta(X, gamma): #update theta component using posteriors (gammas)
        # YOUR CODE HERE
        # --- theta_c: class specific theta, np.array matrix
        # ---        shape: NUM_FEATURES by NUM_MIXTURE_COMPONENTS[c]
        num_mixture_components = gamma.shape[1]
        num_features = X.shape[1]
        theta_c = np.zeros([num_features, num_mixture_components])
        for c in range(num_mixture_components):
            gamma_c = gamma[:, c]
            theta_c[:,c] = np.dot(X.T, gamma_c)/np.sum(gamma_c)
        return theta_c 
    
    @staticmethod
    def findP(X, pi, theta):
        # YOUR CODE HERE
        # --- t: probabilities of x given each component of mixture
        # ---        shape: number of data points in X (where X consists of datapoints from class c) by NUM_MIXTURE_COMPONENTS[c] 
        # --- logsumexp(t,axis=1): normalized by factor of probabilities of x over all components of mixture
        # ---        shape: number of data points in X (where X consists of datapoints from class c)
        
        # Use log-probability for t here
        k = theta.shape[1]
        N = X.shape[0]
        t = np.zeros([N, k])
        for l in range(k):
            t[:,l] = np.log(np.repeat(pi[l], N)) + np.dot(X, np.log(theta[:,l])) + np.dot(1 - X, np.log(1 - theta[:,l]))
        return t,logsumexp(t,axis=1)
        
    # --- execute EM procedure
    def fit(self, X, class_idx):
        max_iter = self.max_iter
        eps = self.epsilon
        N = X.shape[0]
        pi = self.params['pi'][class_idx]
        theta = self.params['theta'][class_idx]
        num_mixture_components = self.num_mixture_components[class_idx]
        # INITIALIZE theta
        for i in range(max_iter):
            # YOUR CODE HERE, E-step: gamma = self.updateLatentPosterior
            # YOUR CODE HERE, M-step(1): pi = self.updatePi 
            # YOUR CODE HERE, M-step(2): theta = self.updateTheta
            gamma = self.updateLatentPosterior(X, pi, theta, num_mixture_components)
            pi = self.updatePi(gamma)
            theta = self.updateTheta(X, gamma)
        return pi,theta #pi and theta, given class_idx

In [116]:
class NaiveBayesModel(AbstractGenerativeModel):
    def __init__(self, CLASSES, NUM_FEATURES, EPS=10**(-12)):
        AbstractGenerativeModel.__init__(self, CLASSES, NUM_FEATURES)
        self.epsilon = EPS # help with stability
        self.params = {
            'logp': [np.zeros((NUM_FEATURES))] * self.num_classes # estimated log-probabilities
        }
    def pack_params(self, X, class_idx):
        logp = self.fit(X[class_idx])
        self.params['logp'][class_idx] = logp
    def classify(self, X): # naive bayes classifier
        # YOUR CODE HERE
        logp = self.params['logp']
        predictions = np.dot(logp, X.T)
        predictions = np.argmax(predictions, axis = 0)
        return predictions
    def fit(self, X): 
        # YOUR CODE HERE
        estimated_logp = np.log(np.mean(X, axis = 0) + self.epsilon)
        return estimated_logp
    
#     def val(self, X, acc=0, N=0):
#         print self.num_classes
#         for c in range(self.num_classes):
#             predictions = self.classify(X[c])
# #             print('predictions: {}, c: {}'.format(predictions, c))
#             print(np.sum(predictions))
#             acc += np.sum((predictions == c).astype(np.int32))
#             N += X[c].shape[0]
#         return (acc / float(N))

In [117]:
experiment_name = "sentiment_analysis"
# --- SENTIMENT ANALYSIS setup
Xtrain,Xval,num_classes,num_features = load_experiment(data_fn, experiment_name)

# -- build naive bayes model for sentiment analysis
print("SENTIMENT ANALYSIS -- NAIVE BAYES MODEL:")
nbm = NaiveBayesModel(num_classes, num_features)
nbm.train(Xtrain)
print("ACCURACY ON VALIDATION: " + str(nbm.val(Xval)))

# -- build mixture model for sentiment analysis
print("SENTIMENT ANALYSIS -- MIXTURE MODEL:")
for i in range(MAX_OUTER_ITER):
    num_mixture_components =  np.random.randint(2,15,num_classes)
    print("COMPONENTS: " + " ".join(str(i) for i in num_mixture_components))
    mm = MixtureModel(num_classes, num_features, num_mixture_components)
    mm.train(Xtrain)
    print("ACCURACY ON VALIDATION: " + str(mm.val(Xval)))

# submit to kaggle
Xkaggle = load_data(data_fn, experiment_name, "kaggle")
save_submission("nb-{}-submission.csv".format(experiment_name), nbm.classify(Xkaggle))
save_submission("mm-{}-submission.csv".format(experiment_name), mm.classify(Xkaggle))

SENTIMENT ANALYSIS -- NAIVE BAYES MODEL:
ACCURACY ON VALIDATION: 0.72
SENTIMENT ANALYSIS -- MIXTURE MODEL:
COMPONENTS: 5 8


KeyboardInterrupt: 

In [5]:
experiment_name = "mnist"
# --- MNIST DIGIT CLASSIFICATION setup
Xtrain,Xval,num_classes,num_features = load_experiment(data_fn, experiment_name)

# -- build naive bayes model for mnist digit classification
print("MNIST DIGIT CLASSIFICATION -- NAIVE BAYES MODEL:")
nbm = NaiveBayesModel(num_classes, num_features)
nbm.train(Xtrain)
print("ACCURACY ON VALIDATION: " + str(nbm.val(Xval)))

# -- build mixture model for mnist digit classification
print("MNIST DIGIT CLASSIFICATION -- MIXTURE MODEL:")
for i in range(MAX_OUTER_ITER):
    num_mixture_components =  np.random.randint(2,15,num_classes)
    print("COMPONENTS: " + " ".join(str(i) for i in num_mixture_components))
    mm = MixtureModel(num_classes, num_features, num_mixture_components)
    mm.train(Xtrain)
    print("ACCURACY ON VALIDATION: " + str(mm.val(Xval)))
    
# submit to kaggle
Xkaggle = load_data(data_fn, experiment_name, "kaggle")
save_submission("nb-{}-submission.csv".format(experiment_name), nbm.classify(Xkaggle))
save_submission("mm-{}-submission.csv".format(experiment_name), mm.classify(Xkaggle))

MNIST DIGIT CLASSIFICATION -- NAIVE BAYES MODEL:
ACCURACY ON VALIDATION: 0.216
MNIST DIGIT CLASSIFICATION -- MIXTURE MODEL:
COMPONENTS: 2 11 2 8 12 5 3 9 11 7


NameError: name 'MixtureModel' is not defined